# Build a variational autoencoder

### References

https://blog.keras.io/building-autoencoders-in-keras.html

https://towardsdatascience.com/understanding-variational-autoencoders-vaes-f70510919f73

In [1]:
import sys
import numpy as np
import keras
from keras import regularizers
from keras import layers
from keras import backend
import random

if '..' not in sys.path:
    sys.path.append('..')
from car_reg_generator.car_reg_generator.uk_reg import UkRegGenerator
from car_reg_generator.car_reg_generator.uk_reg import UkRegBowVectorizer

### Data generation

In [2]:
n_train = 10000
n_val = 1000
n_test = 1000

random.seed(0)
g = UkRegGenerator()
v = UkRegBowVectorizer()

train_strs = [g.get_reg() for _ in range(n_train)]
train_vecs = np.array([v.vectorize(x) for x in train_strs])
val_strs = [g.get_reg() for _ in range(n_val)]
val_strs = [x for x in val_strs if x not in set(train_strs)]  # sanity check
val_vecs = np.array([v.vectorize(x) for x in val_strs])
test_strs = [g.get_reg() for _ in range(n_test)]
test_strs = [x for x in test_strs if x not in set(train_strs) | set(val_strs)]  # sanity check
test_vecs = np.array([v.vectorize(x) for x in test_strs])

vec_length = len(train_vecs[0])
print('input vector length = ' + str(vec_length))
print('actual number of validation samples = ' + str(len(val_strs)))
print('actual number of test samples = ' + str(len(test_strs)))

input vector length = 252
actual number of validation samples = 1000
actual number of test samples = 1000


### Variational autoencoder

Design the architecture

In [3]:
latent_dim = 60
intermediate_dim = 500
original_dim = vec_length

### Define the encoder
inputs = keras.Input(shape=(original_dim,))
h = layers.Dense(intermediate_dim, activation='relu')(inputs)
h = layers.Dense(intermediate_dim, activation='relu')(h)
h = layers.Dense(intermediate_dim, activation='relu')(h)
z_mean = layers.Dense(latent_dim)(h)
z_log_sigma = layers.Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = backend.random_normal(shape=(backend.shape(z_mean)[0], latent_dim),
                                    mean=0., stddev=0.1)
    return z_mean + backend.exp(z_log_sigma) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_sigma])

encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')

### Define the decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = layers.Dense(intermediate_dim, activation='relu')(latent_inputs)
# x = layers.Dense(intermediate_dim, activation='relu')(x)
outputs = layers.Dense(original_dim, activation='sigmoid')(x)
decoder = keras.Model(latent_inputs, outputs, name='decoder')

outputs = decoder(encoder(inputs)[2])
vae = keras.Model(inputs, outputs, name='vae_mlp')

### Define the loss function
reconstruction_loss = keras.losses.binary_crossentropy(inputs, outputs)
reconstruction_loss *= original_dim
kl_loss = 1 + z_log_sigma - backend.square(z_mean) - backend.exp(z_log_sigma)
kl_loss = backend.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = backend.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

Do some training

In [4]:
vae.fit(train_vecs, train_vecs,
        epochs=200,
        batch_size=256,
        shuffle=True,
        validation_data=(val_vecs, val_vecs))

Epoch 1/200
40/40 [==============================] - 3s 70ms/step - loss: 148.3531 - val_loss: 123.0055
Epoch 2/200
40/40 [==============================] - 2s 58ms/step - loss: 109.9998 - val_loss: 99.9280
Epoch 3/200
40/40 [==============================] - 2s 62ms/step - loss: 94.1763 - val_loss: 89.0331
Epoch 4/200
40/40 [==============================] - 3s 65ms/step - loss: 85.7683 - val_loss: 82.6393
Epoch 5/200
40/40 [==============================] - 3s 63ms/step - loss: 80.3173 - val_loss: 77.9673
Epoch 6/200
40/40 [==============================] - 3s 67ms/step - loss: 76.2441 - val_loss: 74.3476
Epoch 7/200
40/40 [==============================] - 3s 69ms/step - loss: 72.7934 - val_loss: 70.8467
Epoch 8/200
40/40 [==============================] - 3s 70ms/step - loss: 69.3609 - val_loss: 67.8662
Epoch 9/200
40/40 [==============================] - 3s 72ms/step - loss: 66.8200 - val_loss: 65.5514
Epoch 10/200
40/40 [==============================] - 3s 70ms/step - loss: 64.6

Epoch 76/200
40/40 [==============================] - 3s 65ms/step - loss: 17.6540 - val_loss: 17.9872
Epoch 77/200
40/40 [==============================] - 3s 65ms/step - loss: 17.4568 - val_loss: 17.9340
Epoch 78/200
40/40 [==============================] - 2s 59ms/step - loss: 17.2932 - val_loss: 17.6197
Epoch 79/200
40/40 [==============================] - 2s 61ms/step - loss: 17.1529 - val_loss: 17.5560
Epoch 80/200
40/40 [==============================] - 3s 63ms/step - loss: 16.9824 - val_loss: 17.3694
Epoch 81/200
40/40 [==============================] - 2s 60ms/step - loss: 16.7869 - val_loss: 17.1492
Epoch 82/200
40/40 [==============================] - 2s 61ms/step - loss: 16.6023 - val_loss: 17.0206
Epoch 83/200
40/40 [==============================] - 2s 59ms/step - loss: 16.4389 - val_loss: 16.8702
Epoch 84/200
40/40 [==============================] - 3s 63ms/step - loss: 16.2754 - val_loss: 16.6737
Epoch 85/200
40/40 [==============================] - 2s 61ms/step - loss

40/40 [==============================] - 3s 63ms/step - loss: 9.7463 - val_loss: 10.3353
Epoch 156/200
40/40 [==============================] - 2s 62ms/step - loss: 9.7198 - val_loss: 10.1257
Epoch 157/200
40/40 [==============================] - 2s 60ms/step - loss: 9.6281 - val_loss: 10.2143
Epoch 158/200
40/40 [==============================] - 2s 61ms/step - loss: 9.5735 - val_loss: 10.1583
Epoch 159/200
40/40 [==============================] - 3s 63ms/step - loss: 9.5422 - val_loss: 9.9851
Epoch 160/200
40/40 [==============================] - 2s 62ms/step - loss: 9.4750 - val_loss: 9.9780
Epoch 161/200
40/40 [==============================] - 2s 57ms/step - loss: 9.4261 - val_loss: 9.9592
Epoch 162/200
40/40 [==============================] - 2s 60ms/step - loss: 9.3760 - val_loss: 9.8233
Epoch 163/200
40/40 [==============================] - 3s 64ms/step - loss: 9.3054 - val_loss: 9.7490
Epoch 164/200
40/40 [==============================] - 2s 60ms/step - loss: 9.2859 - val_los

Evaluation

In [5]:
decoded_regs = vae.predict(test_vecs)
print(test_strs[:10])
print([v.recover(x) for x in decoded_regs[:10]])
acc = np.sum([v.recover(x) == y for x, y in zip(decoded_regs, test_strs)]) / len(test_strs)
print('accuracy = ' + str(acc))

['MY33WSZ', 'WE83GXF', 'LU19JLV', 'KX41GVE', 'WP91NCV', 'SG47LFF', 'HB17WIY', 'CA89AXN', 'GT57AKA', 'CD77SJT']
['MY33WSZ', 'WE83GXF', 'LU19JLV', 'KX41GVE', 'WP97NCV', 'SG47LFF', 'HB17WIY', 'CA89AXN', 'GT57AKA', 'CD77SJT']
accuracy = 0.945


Use decoder as generator

In [6]:
generated_regs = decoder.predict(np.random.random((5, latent_dim)))
print([v.recover(x) for x in generated_regs])

['AS8PU0S', 'AU81BRS', 'AU8AUGS', '5L8ETIA', 'JU81TDS']
